In [1]:
Sys.setenv(TORCH_HOME = Sys.getenv("TORCH_HOME")) 
Sys.setenv(TORCH_INSTALL_CUDA = "1") 
library(torch) 
cuda_is_available() 

[1] TRUE

In [2]:
library(torch)
library(tibble)
library(dplyr)
library(psych)   
library(coro)
library(readr)
library(yardstick)
library(progress)
library(cli)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘coro’


The following object is masked from ‘package:dplyr’:

    collect



Attaching package: ‘yardstick’


The following object is masked from ‘package:readr’:

    spec




In [3]:
data_path <- "/scratch/tmp/ssureshk/torchdata"
RESULTS_DIR   <- "training_logs_r"
dir.create(RESULTS_DIR, showWarnings = FALSE)
BATCH_SIZE <- 1024
EPOCHS <- 11
LEARNING_RATE <- 2.38e-4
WEIGHT_DECAY <- 5.18e-5
DROPOUT_RATE  <- 0.182039
N_CLASSES     <- 9
MODEL_NAME    <- "TempCNN"
class_names <- paste0("Class_", 1:N_CLASSES)

In [4]:
device <- if (cuda_is_available()) torch_device("cuda") else torch_device("cpu")
cat("Using device:", device$type, "\n")

Using device: cuda 


In [5]:
cat("Loading tensors...\n")
X <- torch_load(file.path(data_path, "X_combined.pt"))  # keep on CPU
y <- torch_load(file.path(data_path, "y_combined.pt"))  # keep on CPU

# Sanity checks
stopifnot(X$dim() == 3, X$size(2) == 13, X$size(3) == 45)
stopifnot(y$dim() == 1, y$size(1) == X$size(1))

Loading tensors...


In [6]:
cat("Data shapes:\n")
cat("  X:", paste(X$shape, collapse = " x "), "\n")
cat("  y:", paste(y$shape, collapse = " x "), "\n")

Data shapes:
  X: 485649 x 13 x 45 
  y: 485649 


In [7]:
Bre_Dataset <- dataset(
  name = "BreizhPTDataset",
  initialize = function(X, y) {
    self$X <- X
    self$y <- y
  },
  .getitem = function(i) {
    # Coerce index to a base R integer scalar
    if (inherits(i, "torch_tensor")) {
      i <- as.integer(as_array(i))
    } else {
      i <- as.integer(i)
    }
    if (length(i) != 1L) {
      stop("Dataset .getitem expected a single index, got length = ", length(i))
    }

    # Now safe to slice
    x <- self$X[i, , , drop = FALSE]$squeeze(1)  # [13, 45]
    y <- self$y[i]                                

    list(x = x, y = y)
  },
  .length = function() {
    self$X$size(1)
  }
)


train_ds <- Bre_Dataset(X, y)
train_dl <- dataloader(
  train_ds,
  batch_size = BATCH_SIZE,
  shuffle = TRUE,
  num_workers = 0,    # set >0 if the env supports it
  pin_memory = FALSE  # TRUE can help on CUDA with host->device copies
)

cat("Dataset created with", length(train_ds), "samples\n")
cat("Batches per epoch:", length(train_dl), "\n")

Dataset created with 485649 samples
Batches per epoch: 475 


In [8]:
TempCNN <- nn_module(
  "TempCNN",
  initialize = function(input_channels = 13, n_classes = 9, dropout = 0.182039) {
    self$conv_bn_relu1 <- nn_sequential(
      nn_conv1d(input_channels, 128, kernel_size = 7, padding = 3),
      nn_batch_norm1d(128),
      nn_relu(),
      nn_dropout(p = dropout)
    )
    self$conv_bn_relu2 <- nn_sequential(
      nn_conv1d(128, 128, kernel_size = 7, padding = 3),
      nn_batch_norm1d(128),
      nn_relu(),
      nn_dropout(p = dropout)
    )
    self$conv_bn_relu3 <- nn_sequential(
      nn_conv1d(128, 128, kernel_size = 7, padding = 3),
      nn_batch_norm1d(128),
      nn_relu(),
      nn_dropout(p = dropout)
    )
    self$flatten <- nn_flatten(start_dim = 2)
    self$dense <- nn_sequential(
      nn_linear(128 * 45, 512),
      nn_batch_norm1d(512),
      nn_relu(),
      nn_dropout(p = dropout)
    )
    self$classifier <- nn_linear(512, n_classes)
  },
  forward = function(x) {
    x <- self$conv_bn_relu1(x)
    x <- self$conv_bn_relu2(x)
    x <- self$conv_bn_relu3(x)
    x <- self$flatten(x)
    x <- self$dense(x)
    self$classifier(x)  # logits
  }
)

In [9]:
safe_kappa <- function(y_true, y_pred) {
  k <- tryCatch({
    psych::cohen.kappa(cbind(as.integer(y_true), as.integer(y_pred)))$kappa
  }, error = function(e) NA_real_)
  ifelse(is.na(k), 0, k)
}

calculate_precision_recall_f1 <- function(y_true, y_pred, n_classes, average = "macro") {
  y_true <- factor(y_true, levels = 1:n_classes)
  y_pred <- factor(y_pred, levels = 1:n_classes)
  if (average == "micro") {
    tp <- sum(y_true == y_pred); total <- length(y_true)
    precision <- recall <- f1 <- tp / total
  } else {
    precision_per_class <- numeric(n_classes)
    recall_per_class    <- numeric(n_classes)
    f1_per_class        <- numeric(n_classes)
    for (i in 1:n_classes) {
      tp <- sum(y_true == i & y_pred == i)
      fp <- sum(y_true != i & y_pred == i)
      fn <- sum(y_true == i & y_pred != i)
      precision_per_class[i] <- ifelse(tp + fp == 0, 0, tp / (tp + fp))
      recall_per_class[i]    <- ifelse(tp + fn == 0, 0, tp / (tp + fn))
      denom <- precision_per_class[i] + recall_per_class[i]
      f1_per_class[i]        <- ifelse(denom == 0, 0, 2 * precision_per_class[i] * recall_per_class[i] / denom)
    }
    if (average == "macro") {
      precision <- mean(precision_per_class)
      recall    <- mean(recall_per_class)
      f1        <- mean(f1_per_class)
    } else { # weighted
      support <- as.numeric(table(y_true))
      total_support <- sum(support)
      precision <- sum(precision_per_class * support) / total_support
      recall    <- sum(recall_per_class    * support) / total_support
      f1        <- sum(f1_per_class        * support) / total_support
    }
  }
  list(precision = precision, recall = recall, f1 = f1)
}

calculate_comprehensive_metrics <- function(y_true, y_pred, n_classes, epoch_time, iters_per_sec) {
  micro <- calculate_precision_recall_f1(y_true, y_pred, n_classes, "micro")
  macro <- calculate_precision_recall_f1(y_true, y_pred, n_classes, "macro")
  weighted <- calculate_precision_recall_f1(y_true, y_pred, n_classes, "weighted")
  list(
    accuracy = mean(y_true == y_pred),
    kappa = safe_kappa(y_true, y_pred),
    f1_micro = micro$f1,
    f1_macro = macro$f1,
    f1_weighted = weighted$f1,
    recall_micro = micro$recall,
    recall_macro = macro$recall,
    recall_weighted = weighted$recall,
    precision_micro = micro$precision,
    precision_macro = macro$precision,
    precision_weighted = weighted$precision,
    iters_per_sec = iters_per_sec,
    epoch_time_sec = epoch_time
  )
}

create_per_class_report <- function(y_true, y_pred, n_classes, class_names = NULL) {
  if (is.null(class_names)) class_names <- paste0("Class_", 1:n_classes)
  y_true <- factor(y_true, levels = 1:n_classes)
  y_pred <- factor(y_pred, levels = 1:n_classes)
  report <- data.frame(
    class     = class_names,
    precision = numeric(n_classes),
    recall    = numeric(n_classes),
    f1_score  = numeric(n_classes),
    support   = numeric(n_classes)
  )
  for (i in 1:n_classes) {
    tp <- sum(y_true == i & y_pred == i)
    fp <- sum(y_true != i & y_pred == i)
    fn <- sum(y_true == i & y_pred != i)
    report$precision[i] <- ifelse(tp + fp == 0, 0, tp / (tp + fp))
    report$recall[i]    <- ifelse(tp + fn == 0, 0, tp / (tp + fn))
    denom <- report$precision[i] + report$recall[i]
    report$f1_score[i]  <- ifelse(denom == 0, 0, 2 * report$precision[i] * report$recall[i] / denom)
    report$support[i]   <- sum(y_true == i)
  }
  report
}

# ---------- per-epoch files ----------
epoch_file <- function(epoch, stem) sprintf("%s_epoch_%02d.csv", stem, epoch)
save_confusion <- function(y_true, y_pred, class_names, path) {
  cm <- table(
    Truth     = factor(y_true, levels = 1:length(class_names), labels = class_names),
    Predicted = factor(y_pred, levels = 1:length(class_names), labels = class_names)
  )
  write.csv(as.data.frame.matrix(cm), path)
}

In [10]:
R_training <- function() {
  cat("Starting comprehensive training...\n")
  model <- TempCNN(input_channels = 13, n_classes = N_CLASSES, dropout = DROPOUT_RATE)
  if (cuda_is_available()) model <- model$to(device = torch_device("cuda"))
  criterion <- nn_cross_entropy_loss()
  optimizer <- optim_adamw(model$parameters, lr = LEARNING_RATE, weight_decay = WEIGHT_DECAY)

  training_log <- data.frame()
  start_time <- Sys.time()

  for (epoch in 1:EPOCHS) {
    epoch_start_time <- Sys.time()
    model$train()
    epoch_losses <- c()
    all_preds <- c()
    all_true  <- c()
    batch_count <- 0

    coro::loop(for (batch in train_dl) {
      batch_count <- batch_count + 1

      # Move PER BATCH
      x <- batch$x
      t <- batch$y

      # Ensure target dtype is long
      t <- t$to(dtype = torch_long())

      if (cuda_is_available()) {
        x <- x$to(device = torch_device("cuda"), non_blocking = TRUE)
        t <- t$to(device = torch_device("cuda"), non_blocking = TRUE)
      }

      # Forward
      outputs <- model(x)
      loss <- criterion(outputs, t)

      # Backward
      optimizer$zero_grad()
      loss$backward()
      optimizer$step()

      # Collect metrics on CPU
      epoch_losses <- c(epoch_losses, as.numeric(loss$detach()$cpu()))
      preds <- torch_argmax(outputs$detach(), dim = 2)$cpu()
      all_preds <- c(all_preds, as.integer(preds))
      all_true  <- c(all_true,  as.integer(t$detach()$cpu()))
    })

    epoch_end_time <- Sys.time()
    epoch_duration <- as.numeric(difftime(epoch_end_time, epoch_start_time, units = "secs"))
    iters_per_sec <- batch_count / epoch_duration

    # Epoch-level metrics
    metrics <- calculate_comprehensive_metrics(all_true, all_preds, N_CLASSES, epoch_duration, iters_per_sec)
    metrics$epoch <- epoch
    metrics$loss  <- mean(epoch_losses)

    metrics <- metrics[c(
  "epoch",
  "loss",
  "accuracy",
  "kappa",
  "f1_micro",
  "f1_macro",
  "f1_weighted",
  "recall_micro",
  "recall_macro",
  "recall_weighted",
  "precision_micro",
  "precision_macro",
  "precision_weighted",
  "iters_per_sec",
  "epoch_time_sec"
)]

    # Per-class + confusion per epoch
    per_class_epoch <- create_per_class_report(all_true, all_preds, N_CLASSES, class_names)
    write.csv(per_class_epoch,
              file.path(RESULTS_DIR, epoch_file(epoch, "per_class")),
              row.names = FALSE)

    save_confusion(all_true, all_preds, class_names,
                   file.path(RESULTS_DIR, epoch_file(epoch, "confusion_matrix")))

    # Log 
    message(sprintf(
  "[Epoch %d] Loss=%.4f, Acc=%.4f, F1_macro=%.4f, Kappa=%.4f, %.2f it/s, Time=%.1fs",
  epoch, metrics$loss, metrics$accuracy, metrics$f1_macro,
  metrics$kappa, metrics$iters_per_sec, metrics$epoch_time_sec
))

    training_log <- rbind(training_log, as.data.frame(metrics))
    write.csv(training_log,
              file.path(RESULTS_DIR, paste0(MODEL_NAME, "_log.csv")),
              row.names = FALSE)
  }

  total_time <- as.numeric(difftime(Sys.time(), start_time, units = "secs"))
  cat("Total training time:", round(total_time, 2), "seconds\n")
  list(model = model, history = training_log, training_time = total_time)
}

# ----------------------------
# Run training
# ----------------------------
training_results <- R_training()


Starting comprehensive training...


[Epoch 1] Loss=0.7398, Acc=0.6926, F1_macro=0.4466, Kappa=0.6009, 3.82 it/s, Time=124.5s

[Epoch 2] Loss=0.5433, Acc=0.7632, F1_macro=0.5230, Kappa=0.6932, 3.86 it/s, Time=123.1s

[Epoch 3] Loss=0.5065, Acc=0.7771, F1_macro=0.5372, Kappa=0.7114, 3.87 it/s, Time=122.8s

[Epoch 4] Loss=0.4890, Acc=0.7841, F1_macro=0.5443, Kappa=0.7205, 3.87 it/s, Time=122.6s

[Epoch 5] Loss=0.4750, Acc=0.7902, F1_macro=0.5499, Kappa=0.7285, 3.85 it/s, Time=123.3s

[Epoch 6] Loss=0.4646, Acc=0.7948, F1_macro=0.5538, Kappa=0.7346, 3.87 it/s, Time=122.7s

[Epoch 7] Loss=0.4559, Acc=0.7981, F1_macro=0.5571, Kappa=0.7390, 3.87 it/s, Time=122.9s

[Epoch 8] Loss=0.4479, Acc=0.8012, F1_macro=0.5609, Kappa=0.7430, 3.86 it/s, Time=123.0s

[Epoch 9] Loss=0.4412, Acc=0.8043, F1_macro=0.5639, Kappa=0.7471, 3.88 it/s, Time=122.3s

[Epoch 10] Loss=0.4352, Acc=0.8068, F1_macro=0.5680, Kappa=0.7504, 3.94 it/s, Time=120.5s

[Epoch 11] Loss=0.4289, Acc=0.8095, F1_macro=0.5721, Kappa=0.7539, 3.97 it/s, Time=119.6s



Total training time: 1389.78 seconds


In [ ]:
cat("Saving model...\n")
torch_save(training_results$model, file.path(RESULTS_DIR, "tempcnn_model.pt"))
if (!is.null(training_results$history)) {
  write.csv(training_results$history, file.path(RESULTS_DIR, "training_history.csv"), row.names = FALSE)
}

In [12]:
# ----------------------------
# Final evaluation on full train set
# ----------------------------
cat("Performing final evaluation...\n")
training_results$model$eval()
all_preds <- c(); all_true <- c()

with_no_grad({
  coro::loop(for (batch in train_dl) {
    x <- batch$x
    t <- batch$y$to(dtype = torch_long())
    if (cuda_is_available()) {
      x <- x$to(device = torch_device("cuda"), non_blocking = TRUE)
    }
    out <- training_results$model(x)
    preds_batch <- torch_argmax(out, dim = 2)$cpu()
    all_preds <- c(all_preds, as.integer(preds_batch))
    all_true  <- c(all_true,  as.integer(t$cpu()))
  })
})

final_metrics <- calculate_comprehensive_metrics(all_true, all_preds, N_CLASSES, 0, 0)
final_metrics$samples <- length(all_true)
final_metrics$training_time <- training_results$training_time

write.csv(as.data.frame(final_metrics), file.path(RESULTS_DIR, "final_comprehensive_metrics.csv"), row.names = FALSE)

final_per_class <- create_per_class_report(all_true, all_preds, N_CLASSES, class_names)
write.csv(final_per_class, file.path(RESULTS_DIR, "final_per_class_report.csv"), row.names = FALSE)

final_conf_matrix_path <- file.path(RESULTS_DIR, "final_confusion_matrix.csv")
save_confusion(all_true, all_preds, class_names, final_conf_matrix_path)

cat("\n=== COMPREHENSIVE FINAL RESULTS ===\n")
cat("Accuracy:", round(final_metrics$accuracy, 4), "\n")
cat("Cohen's Kappa:", round(final_metrics$kappa, 4), "\n")
cat("F1 Macro:", round(final_metrics$f1_macro, 4), "\n")
cat("F1 Micro:", round(final_metrics$f1_micro, 4), " (accuracy for the overall samples not per class)\n")
cat("F1 Weighted:", round(final_metrics$f1_weighted, 4), "\n")
cat("Precision Macro:", round(final_metrics$precision_macro, 4), "\n")
cat("Recall Macro:", round(final_metrics$recall_macro, 4), "\n")
cat("\nTraining completed! Files saved to:", RESULTS_DIR, "\n")
cat("\nGenerated files:\n")
cat("- ", MODEL_NAME, "_log.csv (per-epoch comprehensive metrics)\n")
cat("- per_class_epoch_XX.csv (per-epoch per-class metrics)\n")
cat("- confusion_matrix_epoch_XX.csv (per-epoch confusion matrices)\n")
cat("- final_per_class_report.csv\n")
cat("- final_confusion_matrix.csv\n")
cat("- final_comprehensive_metrics.csv\n")
cat("- tempcnn_model.pt\n")

Performing final evaluation...

=== COMPREHENSIVE FINAL RESULTS ===
Accuracy: 0.8148 
Cohen's Kappa: 0.7596 
F1 Macro: 0.5735 
F1 Micro: 0.8148  (accuracy for the overall samples not per class)
F1 Weighted: 0.8046 
Precision Macro: 0.6718 
Recall Macro: 0.5651 

Training completed! Files saved to: training_logs_r 

Generated files:
-  TempCNN _log.csv (per-epoch comprehensive metrics)
- per_class_epoch_XX.csv (per-epoch per-class metrics)
- confusion_matrix_epoch_XX.csv (per-epoch confusion matrices)
- final_per_class_report.csv
- final_confusion_matrix.csv
- final_comprehensive_metrics.csv
- tempcnn_model.pt


Run the trained model on the FRH04 Region

In [ ]:
library(torch)

# Load the model
# Path of the trained model
tempcnn_model <- torch_load("/training_logs_r/tempcnn_model.pt")
tempcnn_model$eval()

# FRH04 Tensor data 
X <- torch_load("/X_combined_4.pt")   
y_true <- torch_load("/y_combined_4.pt") 
# Crop class names mapping
classnames <- read.csv("/classmapping.csv") %>% dplyr::rename(CODE_CULTU = code, class_id = id) %>% dplyr::group_by(class_id) %>% dplyr::summarise(ClassName = first(classname)) %>% dplyr::arrange(class_id) %>% dplyr::pull(ClassName)

# Forward pass
with_no_grad({
  logits <- tempcnn_model(X$to(device = torch_device("cpu")))
})

# Get predicted labels (argmax along class dimension)
y_pred <- torch_max(logits, dim = 2)[[2]]$to(dtype = torch_int()) 

In [ ]:
classification_report_full <- function(y_true, y_pred, classnames = NULL) { # Ensure integer vectors (should be 0-based labels: 0...8) 
  y_true <- as.integer(y_true) 
  y_pred <- as.integer(y_pred)
  classes <- sort(unique(c(y_true, y_pred))) 
  n_classes <- length(classes)
  label_to_index <- setNames(seq_along(classes), classes) 
  y_true_idx <- label_to_index[as.character(y_true)] 
  y_pred_idx <- label_to_index[as.character(y_pred)]
  cm <- matrix(0, nrow = n_classes, ncol = n_classes) 
  for (i in seq_along(y_true_idx)) 
  { cm[y_true_idx[i], y_pred_idx[i]] <- cm[y_true_idx[i], y_pred_idx[i]] + 1 }
  # Per-class metrics 
  metrics <- tibble( ClassID = classes, 
                     Precision = numeric(n_classes), 
                     Recall = numeric(n_classes), 
                     F1 = numeric(n_classes), Support = numeric(n_classes) )
  for (i in 1:n_classes) { 
    tp <- cm[i, i] 
    fp <- sum(cm[, i]) - tp 
    fn <- sum(cm[i, ]) - tp 
    support <- sum(cm[i, ])
    precision <- if ((tp + fp) == 0) 0 else tp / (tp + fp) 
    recall <- if ((tp + fn) == 0) 0 else tp / (tp + fn) 
    f1 <- if ((precision + recall) == 0) 0 else 2 * precision * recall / (precision + recall) 
    metrics$Precision[i] <- precision 
    metrics$Recall[i] <- recall 
    metrics$F1[i] <- f1 
    metrics$Support[i] <- support }
  
  # Add classnames 
 if (!is.null(classnames)) {
   classnames <- unique(classnames) 
   if (length(classnames) == n_classes) { 
     metrics <- metrics %>% mutate(ClassName = classnames[ClassID]) %>% select(ClassID, ClassName, everything()) 
     } 
   else { warning("Number of provided classnames does not match number of unique classes.") } }

  # === Overall metrics ===
  total_correct <- sum(diag(cm))
  total_samples <- sum(cm)
  accuracy <- total_correct / total_samples
  
  macro_avg <- c(
    Precision = mean(metrics$Precision),
    Recall    = mean(metrics$Recall),
    F1        = mean(metrics$F1)
  )
  
  weighted_avg <- c(
    Precision = sum(metrics$Precision * metrics$Support) / sum(metrics$Support),
    Recall    = sum(metrics$Recall * metrics$Support) / sum(metrics$Support),
    F1        = sum(metrics$F1 * metrics$Support) / sum(metrics$Support)
  )
  
  return(list(
    per_class = metrics,
    accuracy = accuracy,
    macro_avg = macro_avg,
    weighted_avg = weighted_avg
  ))
}

In [ ]:
library(readr)
report_r <- classification_report_full(y_true, y_pred, classnames)

# Per-class results with class names
print(report_r$per_class)
write.csv(report_r$per_class, "/r_classification_report.csv", row.names = FALSE)

# Overall
cat("\nOverall Accuracy:", round(report_r$accuracy * 100, 2), "%\n")
cat("Macro Avg:", round(report_r$macro_avg, 3), "\n")
cat("Weighted Avg:", round(report_r$weighted_avg, 3), "\n")

Prediction Results of FRH04 saved to CSV

In [ ]:
library(torch)
library(dplyr)
library(readr)

# Load model
tempcnn_model <- torch_load("/training_logs_r/tempcnn_model.pt")
tempcnn_model$eval()

# Load tensors
X       <- torch_load("/torchdata/X_combined_4.pt")         
y_true  <- torch_load("/torchdata/y_combined_4.pt")         
field_ids <- torch_load("/field_ids_combined_4.pt") 

# Class names (ensure index corresponds to your 1-based labels)
classnames <- read.csv("/classmapping.csv") %>%
  dplyr::rename(CODE_CULTU = code, class_id = id) %>%
  dplyr::group_by(class_id) %>%
  dplyr::summarise(ClassName = dplyr::first(classname), .groups = "drop") %>%
  dplyr::arrange(class_id) %>%
  dplyr::pull(ClassName)

# Forward pass (CPU here; switch to CUDA if available)
with_no_grad({
  logits <- tempcnn_model(X$to(device = torch_device("cpu")))  
})

probs <- nnf_softmax(logits, dim = 2)
mx <- torch_max(probs, dim = 2)
top1_prob <- as.numeric(mx[[1]]$cpu())
y_pred    <- as.integer(mx[[2]]$cpu())



# results table
y_true_r <- as.integer(y_true$to(device = "cpu")$to(dtype = torch_int())$contiguous()$numpy())
y_pred_r <- as.integer(y_pred$to(device = "cpu")$numpy())
fid_r    <- as.integer(field_ids$to(device = "cpu")$numpy())
p1_r     <- as.numeric(top1_prob$to(device = "cpu")$numpy())

df_pred <- tibble::tibble(
  field_id   = fid_r,
  y_true_id  = y_true_r,
  y_true_cls = classnames[y_true_r],
  y_pred_id  = y_pred_r,
  y_pred_cls = classnames[y_pred_r],
  top1_prob  = round(p1_r, 6),
  correct    = (y_true_r == y_pred_r)
)


output_csv <- "/predictions_frh04_with_field_ids.csv"
readr::write_csv(df_pred, output_csv)
cat("Saved:", out_csv, "\n")
